<a href="https://colab.research.google.com/github/arminwitte/llsi/blob/main/notebooks/example_diss_sarimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/arminwitte/llsi.git

fatal: destination path 'llsi' already exists and is not an empty directory.


In [2]:
%cd llsi
# !git checkout frequency_response
!git pull

c:\Users\letsg\git\llsi\notebooks\llsi
Already up to date.


From https://github.com/arminwitte/llsi
   cf3ec32..90de8e1  covariance -> origin/covariance


In [3]:
%pip install -e .

Note: you may need to restart the kernel to use updated packages.


c:\Users\letsg\git\llsi\.venv\Scripts\python.exe: No module named pip


In [4]:
%cd ..

c:\Users\letsg\git\llsi\notebooks


In [5]:
try:
  from llsi import SysIdData  # noqa: F401
except (ImportError, KeyError, ModuleNotFoundError, AttributeError):
  print('Stopping RUNTIME to reload. Please run again.')
  exit()

c:\Users\letsg\git\llsi\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note: restart runtime after first install run!

In [6]:
import matplotlib.pyplot as plt
import numpy as np

import llsi

In [7]:
d = np.load("llsi/data/heated_wire_data.npy")

In [ ]:
t = d[:, 0]
Re = d[:, 1]
Nu = d[:, 2]

: 

In [ ]:
data = llsi.SysIdData(t=t, Re=Re, Nu=Nu)
data.equidistant(305002)
print(data.time().shape)
data.center()
data.downsample(18)
data.lowpass(1, 10)
data, test_set = data.split(0.8)
data.crop(start=1000)
data.center()
print(1 / data.Ts)

: 

In [ ]:
# mod1 = llsi.sysid(data, "Nu", "Re", (0, 200, 0), method="arx", settings={"lambda": 2e1})
mod1 = llsi.sysid(data, "Nu", "Re", (2, 2, 0), method="arx", settings={"lambda": 1e-6})
mod2 = llsi.sysid(
    data,
    "Nu",
    "Re",
    3,
    method="pem",
    settings={"init": "po-moesp", "minimizer_kwargs": {"method": "BFGS"}},
)


: 

In [ ]:
with llsi.Figure() as fig:
    fig.plot([mod1, mod2], "impulse")
    fig.plot([mod1, mod2], "frequency")
    fig.plot(
        {"mod": [mod1, mod2], "data": test_set, "y_name": "Nu", "u_name": "Re"},
        "compare",
    )

: 

In [ ]:
fig, ax = plt.subplots()
ti1, i1 = mod1.impulse_response(300)
ti2, i2 = mod2.impulse_response(300)
plt.plot(ti1[50:], i1[50:])
plt.plot(ti2[50:], i2[50:], "r")
plt.grid(True)

print(i1[:5])
# print(i2[:5])

: 

## Comparison with statsmodels SARIMAX

We will now train a SARIMAX model using `statsmodels` and compare its performance with the `llsi` models on the test set.

**Note on SARIMAX Performance:**
Standard `SARIMAX` treats exogenous variables (`exog`) as static regressors. To capture the dynamic response (transfer function) of the system, we explicitly construct a matrix of lagged inputs (a Finite Impulse Response or FIR basis) and pass it as `exog`. We will use **3 lags** as requested.

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Prepare training data
endog_train = data["Nu"]
exog_train = data["Re"]

# Prepare test data
endog_test = test_set["Nu"]
exog_test = test_set["Re"]
t_test = test_set.time()

# --- FIX: Create lagged exogenous variables ---
# SARIMAX treats exog as static. We need to manually add lags to model dynamics.
def create_lagged_exog(u, lags):
    N = len(u)
    X = np.zeros((N, lags + 1))
    for i in range(lags + 1):
        # Shift u by i
        X[i:, i] = u[:N-i]
    return X

# Use 3 lags as requested
n_lags = 2
print(f"Creating {n_lags} lagged features for exogenous variable...")
exog_train_lagged = create_lagged_exog(exog_train, n_lags)
exog_test_lagged = create_lagged_exog(exog_test, n_lags)

# Define and fit SARIMAX model
# Using order (2, 0, 2) for the noise/internal dynamics
print("Training SARIMAX...")
mod_sarimax = sm.tsa.statespace.SARIMAX(
    endog_train,
    exog=exog_train_lagged,
    order=(2, 0, 2),
    trend='c'
)
res_sarimax = mod_sarimax.fit(disp=False)
print(res_sarimax.summary())

# Predict with SARIMAX
start_idx = len(endog_train)
end_idx = start_idx + len(endog_test) - 1
pred_sarimax = res_sarimax.predict(
    start=start_idx,
    end=end_idx,
    exog=exog_test_lagged
)

# Simulate llsi models on test set
y_mod1 = mod1.simulate(exog_test)
y_mod2 = mod2.simulate(exog_test)

# Plot comparison
plt.figure(figsize=(14, 7))
plt.plot(t_test, endog_test, label='True Data', color='black', alpha=0.5)
plt.plot(t_test, pred_sarimax, label=f'SARIMAX (2,0,2) + {n_lags} lags', linestyle='--')
plt.plot(t_test, y_mod1, label='llsi ARX', linestyle='-.')
plt.plot(t_test, y_mod2, label='llsi PEM', linestyle=':')

plt.legend()
plt.title('Model Comparison on Test Set')
plt.xlabel('Time')
plt.ylabel('Nu')
plt.grid(True)
plt.show()

# Calculate fits (NRMSE)
def nrmse(y_true, y_pred):
    return 1 - np.linalg.norm(y_true - y_pred) / np.linalg.norm(y_true - np.mean(y_true))

fit_sarimax = nrmse(endog_test, pred_sarimax)
fit_mod1 = nrmse(endog_test, y_mod1.ravel())
fit_mod2 = nrmse(endog_test, y_mod2.ravel())

print(f"Fits (NRMSE):")
print(f"SARIMAX: {fit_sarimax:.4f}")
print(f"llsi ARX: {fit_mod1:.4f}")
print(f"llsi PEM: {fit_mod2:.4f}")

: 

: 